In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials

import time
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import matplotlib
%matplotlib inline

In [ ]:
# Create a client
credentials = CognitiveServicesCredentials(None)
client = ComputerVisionClient(
    endpoint="https://uksouth.api.cognitive.microsoft.com/",
    credentials=credentials
)

In [ ]:
# Open the image as bytes
with open("images/lazy_dog.jpg", 'rb') as image_bytes:

    # Call API
    job = client.read_in_stream(
        image=image_bytes,
        raw=True
    )

In [ ]:
# Get the ID from the URL
operation_id = job.headers["Operation-Location"].split("/")[-1]

# Call the "GET" API and wait until we get a result
while True:
    result = client.get_read_result(operation_id).as_dict()
    if result['status'] not in ['notStarted', 'running']:
        break
    time.sleep(1)    

# Filter to read_result
read_result = result['analyze_result']['read_results'][0]

In [ ]:
# Examine the read result
read_result

In [ ]:
# Print the detected text
for line in read_result["lines"]:
    print(line["bounding_box"])
    print(line["text"])

In [ ]:
# Display the image and overlay it with the extracted text.
image = cv2.imread("images/lazy_dog.jpg")
ax = plt.imshow(image)

for line in read_result["lines"]:
    
    # Extract the (x,y) coordinates from [x0,y0,...,x3,y3]
    bbox = line["bounding_box"]
    vertices = [(bbox[i], bbox[i+1]) for i in range(0, 8, 2)]
    
    # Draw on the polygon
    patch = Polygon(vertices, closed=True, fill=False, linewidth=2, color='y')
    ax.axes.add_patch(patch)
    
    # Add the text
    text = line["text"]
    plt.text(vertices[0][0], vertices[0][1], text, fontsize=20, va="top")

plt.show()